In [3]:
import tensorflow as tf
import tensorflow as tf
import keras
from keras.layers import Conv2D,MaxPooling2D,Dropout
import os
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    c = BatchNormalization()(c)
    c = Activation("relu")(c)
    p = MaxPooling2D((2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = UpSampling2D((2, 2))(x)
    concat = Concatenate()([us, skip])
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    c = BatchNormalization()(c)
    c = Activation("relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    c = BatchNormalization()(c)
    c = Activation("relu")(c)
    return c

def UNet(image_size):
    f = [64, 128, 256, 512, 1024]
    
    inputs = Input((image_size, image_size, 1))
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) 
    c2, p2 = down_block(p1, f[1]) 
    c3, p3 = down_block(p2, f[2]) 
    c4, p4 = down_block(p3, f[3]) 
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) 
    u2 = up_block(u1, c3, f[2]) 
    u3 = up_block(u2, c2, f[1]) 
    u4 = up_block(u3, c1, f[0])
    
    outputs = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = Model(inputs, outputs)
    return model

# Example usage:
image_size = 256
model = UNet(image_size)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [5]:
dataset_path = "Brain_Dataset"
images_path = "Brain_Dataset/data/images"
masks_path = "Brain_Dataset/data/masks"
target_shape = (256,256)
def preprocess_dataset(images_path,masks_path):
    images = []
    masks = []

    for image_file in os.listdir(images_path):
        image_path = os.path.join(images_path, image_file)
        if os.path.isfile(image_path):
            image = cv2.imread(image_path, 0) 
            image = cv2.resize(image, target_shape) 
            images.append(image)

    for mask_file in os.listdir(masks_path):
        mask_path = os.path.join(masks_path, mask_file)
        if os.path.isfile(mask_path):
            mask = cv2.imread(mask_path, 0) 
            mask = cv2.resize(mask, target_shape) 
            masks.append(mask)

    images = np.array(images)
    masks = np.array(masks)

    images = images/255.0
    masks = masks/255.0

    return images,masks

images,masks = preprocess_dataset(images_path,masks_path)

In [6]:
images_train, images_test, masks_train, masks_test = train_test_split(images, masks, test_size= 0.1, random_state=42)
images_train, images_valid, masks_train, masks_valid = train_test_split(images_train, masks_train, test_size= 0.222, random_state=42)

In [10]:
images_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.02352941, 0.01568627, 0.02352941, 0.01568627,
       0.01176471, 0.01960784, 0.02745098, 0.02745098, 0.03137255,
       0.02352941, 0.01960784, 0.01568627, 0.01568627, 0.01568627,
       0.07058824, 0.18823529, 0.37647059, 0.54509804, 0.56862745,
       0.4627451 , 0.3372549 , 0.3254902 , 0.36078431, 0.2745098 ,
       0.21960784, 0.28627451, 0.28235294, 0.17647059, 0.10196078,
       0.13333333, 0.19607843, 0.20784314, 0.21176471, 0.22352941,
       0.23137255, 0.23529412, 0.22352941, 0.21568627, 0.21960

In [8]:
model = UNet(256)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 256, 256, 64  640         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_20 (Conv2D)             (None, 256, 256, 64  36928       ['conv2d_19[0][0]']              
                                )                                                           

In [11]:
def pixelwise_accuracy(y_true, y_pred):
    y_pred_binary = tf.cast(y_pred > 0.5, tf.float32)
    correct_pixels = tf.cast(tf.equal(y_true, y_pred_binary), tf.float32)
    accuracy = tf.reduce_mean(correct_pixels)
    return accuracy

# Intersection over Union (IoU)
def iou(y_true, y_pred):
    y_pred_binary = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred_binary)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred_binary) - intersection
    iou = intersection / union
    return iou

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]


In [12]:
epochs = 150
batch_size = 8

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[pixelwise_accuracy, iou])

history = model.fit(images_train, masks_train, validation_data=(images_valid, masks_valid), epochs=epochs, batch_size=batch_size)

Epoch 1/150
 33/268 [==>...........................] - ETA: 58:09 - loss: 0.1972 - pixelwise_accuracy: 0.9671 - iou: 0.0028